<a href="https://colab.research.google.com/github/Zillth/Evolutionary-algorithm/blob/main/Red_Neurnal_MONIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Técnicas para ampliar el dataset:

Repetición de datos

In [ ]:
import pandas as pd

# Cargar el CSV existente
csv_path = 'facturas.csv'
df = pd.read_csv(csv_path)

# Definir la cantidad de veces que deseas repetir los datos
replication_factor = 5  # Puedes ajustar este valor según tus necesidades

# Repetir los datos en el DataFrame
df_repeated = pd.concat([df] * replication_factor, ignore_index=True)

# Guardar el DataFrame con datos repetidos
df_repeated.to_csv('facturas_rep.csv', index=False)

Creación de datos sintéticos

In [ ]:
!pip install faker

In [ ]:
import pandas as pd
from faker import Faker
import random

# Cargar el CSV existente
csv_path = 'facturas.csv'
df = pd.read_csv(csv_path)

# Inicializar el generador de datos sintéticos
faker = Faker()

faker = Faker('es_ES')  # Establece el idioma español

# Definir la cantidad de muestras sintéticas que deseas generar
num_samples_to_generate = 1000

# Función para generar una fila de datos sintéticos
def generate_synthetic_row():
    return {
        'Cliente': faker.company(),
        'Fecha': faker.date_this_decade(),
        'Monto': random.uniform(1, 1000),
        # INCOMPLETO
    }

# Generar datos sintéticos y agregarlos al DataFrame
for _ in range(num_samples_to_generate):
    synthetic_row = generate_synthetic_row()
    df = df.append(synthetic_row, ignore_index=True)

# Guardar el DataFrame actualizado con las muestras sintéticas
df.to_csv('facturas_fake.csv', index=False)

Datos sinteticos usando Transformadores

In [ ]:
pip install transformers torch

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Cargar modelo pre-entrenado y tokenizador
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Cantidad de muestras sintéticas que deseas generar
num_samples_to_generate = 1000

# Función para generar texto sintético
def generate_synthetic_text(prompt=""):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)
    synthetic_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return synthetic_text

# Generar texto sintético y agregarlo al conjunto de datos
for _ in range(num_samples_to_generate):
    synthetic_text = generate_synthetic_text()
    # Agregar synthetic_text a tu conjunto de datos como sea apropiado INCOMPLETO

# Guardar el conjunto de datos actualizado con datos sintéticos
df.to_csv('facturas.csv', index=False)

Red Neuronal

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar datos desde CSV
facturas_data = pd.read_csv('facturas.csv')
movimientos_data = pd.read_csv('movimientos.csv')

# Combinar datos
merged_data = pd.merge(facturas_data, movimientos_data, how='inner', on='ID_comun')

# Separar características (X) y etiquetas (y)
X = merged_data.drop(['ID_comun', 'fecha', 'nombre', 'clabe'], axis=1)
y = merged_data['isMatched']

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definir el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, input_dim=X_train.shape[1], activation='relu'),  # Capa oculta con 64 neuronas y función de activación ReLU
    tf.keras.layers.Dropout(0.5), Dropout para regularización
    tf.keras.layers.Dense(units=1, activation='sigmoid')  # Capa de salida con 1 neurona y función de activación sigmoide
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Hacer predicciones
predictions = model.predict(X_test)